In [217]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import random
from pprint import pprint


In [218]:
def train_test_split(df, size):
    
    if isinstance(size, float):
        size = round(size * len(df))

    indices = df.index.tolist()
    test_indices = random.sample(population=indices, k=size)

    train_df = df.loc[test_indices]
    test_df = df.drop(test_indices)
    
    return train_df, test_df

In [219]:
def check_purity(data):
    
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False

In [220]:
def classify_data(data):
    
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification

In [221]:
def get_potential_splits(data):
    
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1):          # excluding the last column which is the label
        values = data[:, column_index]
        unique_values = np.unique(values)
        
        potential_splits[column_index] = unique_values
    
    return potential_splits

In [222]:
def split_data(data, split_column, split_value):
    
    split_column_values = data[:, split_column]

    # features are categorical   
    data_below = data[split_column_values == split_value]
    data_above = data[split_column_values != split_value]
    
    return data_below, data_above

In [223]:
def calculate_entropy(data):
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

In [224]:
def calculate_overall_entropy(data_below, data_above):
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_entropy =  (p_data_below * calculate_entropy(data_below) 
                      + p_data_above * calculate_entropy(data_above))
    
    return overall_entropy

In [225]:
def determine_best_split(data, potential_splits):
    
    overall_entropy = 9999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

In [226]:
def decision_tree_algorithm(df, counter=0, min_samples=2, max_depth=5):
    
    # data preparations
    if counter == 0:
        global COLUMN_HEADERS
        COLUMN_HEADERS = df.columns
        data = df.values
    else:
        data = df           
    
    
    # base cases
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        
        return classification

    
    # recursive part
    else:    
        counter += 1
 
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        # check for empty data
        if len(data_below) == 0 or len(data_above) == 0:
            classification = classify_data(data)
            return classification
        
        # determine question
        feature_name = COLUMN_HEADERS[split_column]   
        question = "{} = {}".format(feature_name, split_value)
        
        # instantiate sub-tree
        sub_tree = {question: []}
        
        # find answers (recursion)
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples, max_depth)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples, max_depth)
        
        # If the answers are the same, then there is no point in asking the qestion.
        # This could happen when the data is classified even though it is not pure
        # yet (min_samples or max_depth base case).
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [227]:
def classify_example(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")

    # ask question
    
    if example[int(feature_name)] == value:
        answer = tree[question][0]
    else:
        answer = tree[question][1]

    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return classify_example(example, residual_tree)

In [228]:
def calculate_accuracy(df, tree):

    df["classification"] = df.apply(classify_example, axis=1, args=(tree,))
    df["classification_correct"] = df["classification"] == df["lable"]
    
    accuracy = df["classification_correct"].mean()
    
    return accuracy

In [229]:
df = pd.read_csv("house-votes.csv" ,header=None)
lable = df[0]
df = df.drop(0,axis=1)
df['lable'] = lable 
df.head()

for i in range(1 , len(df.columns) - 1):
  mode = df[i].mode()[0]
  df = df.replace('?', mode)

# random.seed(0)
# train_df, test_df = train_test_split(df,20)

# tree = decision_tree_algorithm(train_df, max_depth=7)
# pprint(tree)

# # example = test_df.iloc[2]
# # example

# accuracy = calculate_accuracy(test_df, tree)
# accuracy

{'4 = y': [{'3 = y': [{'16 = y': ['republican', 'democrat']}, 'republican']},
           {'8 = y': ['democrat', {'9 = y': ['democrat', 'republican']}]}]}


0.8939759036144578

In [230]:
def experimentSplitsData(data , trainSize):
  experiments = []
  for i in range(5):
    # random.seed(i)
    train_df, _ = train_test_split(df,trainSize)
    tree = decision_tree_algorithm(train_df, max_depth=8)
    accuracy = calculate_accuracy(test_df, tree)
    experiments.append((trainSize , accuracy))
  return experiments


In [232]:
train_sizes = [0.25 , 0.30 , 0.40 , 0.50 , 0.60 , 0.70]
experiments = []
for train_size in train_sizes:
  experiments.append(experimentSplitsData(df,train_size))

experiments

[[(0.25, 0.946987951807229),
  (0.25, 0.9686746987951808),
  (0.25, 0.963855421686747),
  (0.25, 0.9566265060240964),
  (0.25, 0.9734939759036144)],
 [(0.3, 0.9590361445783132),
  (0.3, 0.9614457831325302),
  (0.3, 0.9493975903614458),
  (0.3, 0.963855421686747),
  (0.3, 0.9710843373493976)],
 [(0.4, 0.9662650602409638),
  (0.4, 0.9710843373493976),
  (0.4, 0.9686746987951808),
  (0.4, 0.9710843373493976),
  (0.4, 0.9686746987951808)],
 [(0.5, 0.9759036144578314),
  (0.5, 0.944578313253012),
  (0.5, 0.9710843373493976),
  (0.5, 0.9759036144578314),
  (0.5, 0.963855421686747)],
 [(0.6, 0.9759036144578314),
  (0.6, 0.9855421686746988),
  (0.6, 0.9783132530120482),
  (0.6, 0.983132530120482),
  (0.6, 0.9855421686746988)],
 [(0.7, 0.983132530120482),
  (0.7, 0.9879518072289156),
  (0.7, 0.980722891566265),
  (0.7, 0.9855421686746988),
  (0.7, 0.9855421686746988)]]